In [12]:
cd /home/hoseung/Work/data/29172

/media/hoseung/DATA/29172


In [34]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
# Need a full tree. not only main progenitor tree. 
tt = pickle.load(open("./GalaxyMaker/Trees/extended_tree.pickle", "rb"))
from tree import ctutils as ctu
from galaxy.galaxy import Galaxy
import tree.halomodule as hmo
#alltree = extract_a_tree(tt, idx=)

In [15]:
tt.data["nout"]
t_final = tt.data[tt.data["nout"] == 187]

In [16]:
t_final["id"]

array([43176, 43045, 43039, 43038, 43180, 43082, 43078, 43056, 43154,
       43076, 43085, 43051, 43059, 43079, 43103, 43080, 43050, 43070,
       43096, 43130, 43048, 43104, 43090, 43142, 43107, 43113, 43036,
       43074, 43172, 43160, 43092, 43126, 43129, 43065, 43046, 43091,
       43120, 43135, 43168, 43071, 43049, 43153, 43054, 43117, 43044,
       43072, 43064, 43088, 43164, 43106, 43110, 43053, 43083, 43112,
       43052, 43141, 43143, 43061, 43087, 43067, 43119, 43167, 43111,
       43116, 43190, 43069, 43093, 43121, 43124, 43122, 43114, 43134,
       43041, 43185, 43139, 43062, 43099, 43146, 43097, 43105, 43128,
       43140, 43147, 43125, 43101, 43148, 43186, 43150, 43137, 43115,
       43095, 43162, 43173, 43133, 43123, 43188, 43073, 43102, 43131,
       43158, 43089, 43161, 43086, 43075, 43127, 43077, 43047, 43183,
       43157, 43118, 43151, 43109, 43163, 43165, 43136, 43156, 43166,
       43175, 43144, 43152, 43149, 43170, 43155, 43159, 43169])

In [17]:
atree = ctu.extract_a_tree(tt.data, 43045)
maintree = ctu.extract_main_tree(atree, 43045)

allbranches=[]

In [18]:
while len(atree) > 10:
    maintree = ctu.extract_main_tree(atree, atree["id"][0], no_subset=True)
    # Need no_subset=True.
    # A subset of the whole tree is taken by default to reduce the search time. 
    # But the subset if extracted by checking the "tree_root_id".
    # Because the root of each branch is not the root of the tree, 
    # subset shouldn't be taken. 
    allbranches.append(maintree)
    
    # mask extracted tree
    for gal in atree:
        if gal["id"] in maintree["id"]:
            #print(gal["id"])
            gal["id"] = 1 
    
    # keep the rest
    atree = atree[atree["id"] != 1]

In [19]:
mainbranch = allbranches[0]
for branch in allbranches[1:]:
    for gal in branch:
        i = np.where(mainbranch["nout"] == gal["nout"])[0]
        gal["x"] = gal["x"] - mainbranch["x"][i]
        gal["y"] = gal["y"] - mainbranch["y"][i]
        gal["z"] = gal["z"] - mainbranch["z"][i]

In [20]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

for branch in allbranches[1:]:
    ax.scatter(branch["x"], branch["y"], branch["z"], s=branch["m"]/1e8)

ax.scatter(0,0,0,s=200, c='r')
plt.show()

branches are separated.
So far, so good.

Now, spin direction of each galaxy

In [21]:
import load

In [22]:
branch.dtype

dtype([('aexp', '<f8'), ('id', '<i8'), ('desc_aexp', '<i8'), ('desc_id', '<i8'), ('nprog', '<i8'), ('pid', '<i8'), ('upid', '<i8'), ('desc_pid', '<i8'), ('phantom', '<i8'), ('sam_mvir', '<f8'), ('mvir', '<f8'), ('rvir', '<f8'), ('rs', '<i8'), ('vrms', '<f8'), ('mmp', '<f8'), ('aexp_last_MM', '<f8'), ('vmax', '<i8'), ('x', '<f8'), ('y', '<f8'), ('z', '<f8'), ('vx', '<f8'), ('vy', '<f8'), ('vz', '<f8'), ('jx', '<f8'), ('jy', '<f8'), ('jz', '<f8'), ('spin', '<f8'), ('b_id', '<i8'), ('d_id', '<i8'), ('tree_root_id', '<i8'), ('Orig_halo_id', '<i8'), ('nout', '<i8'), ('next_coprogenitor_d_id', '<i8'), ('last_progenitor_d_id', '<i8'), ('last_mainleaf_depthfirst_id', '<i8'), ('tidal_force', '<f8'), ('tidal_id', '<i8'), ('np', '<i4'), ('r', '<f4'), ('tvir', '<f4'), ('cvel', '<f4'), ('sig', '<f4'), ('sigbulge', '<f4'), ('mbulge', '<f4'), ('m', '<f4')])

In [23]:
branch = allbranches[0]
igal = 0
idgal = branch["Orig_halo_id"][igal]
nout = branch["nout"][igal]

In [24]:
info = load.info.Info(nout)

gals = hmo.Halo(nout=nout, is_gal=True)
gg = gals.data[gals.data["id"] == idgal]
gm = load.rd_GM.rd_gal(nout, idgal)
gal = Galaxy(halo=gg, info=info)
good_gal = gal.mk_gal(gm.star, gm.dm, gm.cell,
                      unit_conversion="GM",
                      verbose=False, debug=False)

[Halo.load_info] loading info
[Halo.load_info] nout = 187, base =./
[rd_GM.rd_gal] fname= ./GalaxyMaker/GAL_00187/gal_stars_0000011


/home/hoseung/Work/pyclusterevol/galaxy/galaxy.py:135: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  m_radial[i] = np.sum(m_sorted[ibins[i]:ibins[i+1]])


In [25]:
for branch in allbranches:
    plt.plot(branch["nout"])
             
plt.show()

In [51]:
allbranches[0]

array([ (1.0, 43045, -1, -1, 1, -1, -1, -1, 0, 90690000000.0, 90690000000.0, 45.78, -1, 0.0, 1.0, 0.83, -1, 102.32649, 137.95865, 90.97453, -335.96, 72.34, 154.96, 132600000000000.0, -31850000000000.0, -31030000000000.0, 0.5585, 125, 125, 43045, 11, 187, -1, 456, 291, 0.20672, 43176, 137386, 0.00022890183026902378, 164213.734375, 67.63282012939453, 133.52041625976562, 134.67550659179688, 0.6006125211715698, 63846969344.0),
       (0.995, 42875, -1, 43045, 1, -1, -1, -1, 0, 90496700000.0, 90510000000.0, 46.783, -1, 0.0, 1.0, 0.83, -1, 102.34354, 137.95457, 90.96731, -344.36, 83.44, 126.31, 132500000000000.0, -34350000000000.0, -31410000000000.0, 0.56145, 126, 126, 43045, 8, 186, -1, 456, 291, 0.20218, 43006, 136422, 0.0002339125785510987, 164694.859375, 67.73182678222656, 132.74484252929688, 133.595458984375, 0.6017123460769653, 63400214528.0),
       (0.99, 42696, -1, 42875, 1, -1, -1, -1, 0, 90276700000.0, 90290000000.0, 45.025, -1, 0.0, 1.0, 0.83, -1, 102.36137, 137.95015, 90.96169, 

In [27]:
branch_p = allbranches[0]
branch_s = allbranches[1]

# intersect1d automatically sort the result. 
nouts_common = branch_s["nout"]

gcats = [(hmo.Halo(nout=nout, is_gal=True), nout) for nout in nouts_common]

[Halo.load_info] loading info
[Halo.load_info] nout = 153, base =./
[Halo.load_info] loading info
[Halo.load_info] nout = 152, base =./
[Halo.load_info] loading info
[Halo.load_info] nout = 151, base =./
[Halo.load_info] loading info
[Halo.load_info] nout = 150, base =./
[Halo.load_info] loading info
[Halo.load_info] nout = 149, base =./
[Halo.load_info] loading info
[Halo.load_info] nout = 148, base =./
[Halo.load_info] loading info
[Halo.load_info] nout = 147, base =./
[Halo.load_info] loading info
[Halo.load_info] nout = 146, base =./
[Halo.load_info] loading info
[Halo.load_info] nout = 145, base =./
[Halo.load_info] loading info
[Halo.load_info] nout = 144, base =./
[Halo.load_info] loading info
[Halo.load_info] nout = 143, base =./
[Halo.load_info] loading info
[Halo.load_info] nout = 142, base =./
[Halo.load_info] loading info
[Halo.load_info] nout = 141, base =./
[Halo.load_info] loading info
[Halo.load_info] nout = 140, base =./
[Halo.load_info] loading info
[Halo.load_info] n

In [32]:
from scipy.signal import argrelmin

# pericentric distance = first local minimum in distance

# position of All branches are already in the primary galaxy's rest frame.
dd = np.sqrt(branch_s["x"]**2 + branch_s["y"]**2 + branch_s["z"]**2)

# position of the pericenter
i_peri = argrelmin(dd)[-1].squeeze()
nout_peri = nouts_common[i_peri]

info = load.info.Info(nout_peri)

gals = hmo.Halo(nout=nout_peri, is_gal=True)

# primary galaxy
idgal_pri = branch_p["Orig_halo_id"][nout==nout_peri]
gg = gals.data[gals.data["id"] == idgal_pri]
gm = load.rd_GM.rd_gal(nout_peri, idgal)
gal_p = Galaxy(halo=gg, info=info)
good_gal = gal_p.mk_gal(gm.star, gm.dm, gm.cell,
                  unit_conversion="GM",
                  verbose=False, debug=False)

# secondary galaxy
idgal_sec = branch_s["Orig_halo_id"][nout==nout_peri]
gg = gals.data[gals.data["id"] == idgal_sec]
gm = load.rd_GM.rd_gal(nout_peri, idgal_sec)
gal_s = Galaxy(halo=gg, info=info)
good_gal = gal_s.mk_gal(gm.star, gm.dm, gm.cell,
                  unit_conversion="GM",
                  verbose=False, debug=False)

[Halo.load_info] loading info
[Halo.load_info] nout = 140, base =./


/home/hoseung/mypy/lib/python3.5/site-packages/ipykernel/__main__.py:18: VisibleDeprecationWarning: using a boolean instead of an integer will result in an error in the future
/home/hoseung/Work/pyclusterevol/galaxy/galaxy.py:135: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  m_radial[i] = np.sum(m_sorted[ibins[i]:ibins[i+1]])
/home/hoseung/mypy/lib/python3.5/site-packages/ipykernel/__main__.py:27: VisibleDeprecationWarning: using a boolean instead of an integer will result in an error in the future


[rd_GM.rd_gal] fname= ./GalaxyMaker/GAL_00140/gal_stars_0000011
[rd_GM.rd_gal] fname= ./GalaxyMaker/GAL_00140/gal_stars_0000103


In [36]:
# Galaxy rotation axis at the pericenter.

gal_p.cal_norm_vec()
gal_s.cal_norm_vec()
nvec_pri = gal_p.meta.nvec
nvec_sec = gal_s.meta.nvec


# Angle between two galaxies
# angle between orbital plane - spin of primary galaxy
p_pri = np.array([gal_p.meta.xc, gal_p.meta.yc, gal_p.meta.zc]).ravel()
p_sec = np.array([gal_s.meta.xc, gal_s.meta.yc, gal_s.meta.zc]).ravel()

v_pri = np.array([gal_p.meta.vxc, gal_p.meta.vyc, gal_p.meta.vzc])
v_sec = np.array([gal_s.meta.vxc, gal_s.meta.vyc, gal_s.meta.vzc])

rvec = p_pri - p_sec
vvec = v_pri - v_sec

In [43]:
plt.scatter(branch_p["x"],branch_p["y"])
plt.scatter(branch_s["x"],branch_s["y"])

plt.title("Orbits of the primary galaxy, and a secondary galaxy")
plt.show()


In [35]:
nvec_orbit = np.cross(rvec, vvec)

# angle between orbital plane - spin of secondary galaxy
ang_pri_orb = np.arccos(np.dot(nvec_pri, nvec_orbit) / )
ang_sec_orb = np.arccos(np.dot(nvec_sec, nvec_orbit) / )

# Determine orbital parameters ()
# Orbital eccentricity / pericentric distance / orbit type (para, rad, ellip, hyper)
# How...??


# Special focus on values at the pericenter.

SyntaxError: invalid syntax (<ipython-input-35-a6611bcc26ba>, line 21)

In [177]:
nvec_orbit **2 

array([  22151.51393363,    5618.21298476,  390072.02906411])

In [175]:
nvec_pri * nvec_pri

array([ 0.05671546,  0.00491611,  0.93836844])

In [174]:
np.dot(nvec_pri, nvec_orbit) / np.

564.30495208025513

In [173]:
ang_pri_orb

nan

In [ ]:
def get_merging_system():
    
    
    

In [ ]:
def find_merger_epochs(alltrees,
                       idx_all,
                       mpgs,
                       nout_ini=37,
                       dist_gal_scale_in=2.0,
                       dist_gal_scale_out =3.0,
                       min_mass_ratio = 0.05,
                       mass_ratio="early",
                       verbose=False,
                       do_plot = False,
                       pdf_fname='./merger_ratio.pdf',
                       max_rgal=40):
    """
    Parameters
    ----------
    dist_gal_scale 
        if two galaxies are closer than dist_gal_scale * (sum of raidus of the two),
        that epoch is the nout_init_merger.
    nout_ini
        blabla
    """
    gal_list=[]
    mr_list=[]
    nout_list=[]
    nout_ini_list=[] # initial time when two halos(Galaxy stellar components in this case) overlap. 

    #for idx in idx_all:
    if do_plot:
        from matplotlib.backends.backend_pdf import PdfPages
        fig, ax = plt.subplots(1, sharex=True)
        pdf = PdfPages(pdf_fname)
    else:
        ax = None

    for gal in mpgs:
        idx = gal.data['idx'][0]
        # full tree of a galaxy
        atree = ctu.extract_a_tree(alltrees.data, idx)

        # main progenitor tree
        main = ctu.extract_main_tree(atree, idx)
        main_nout = main['nout'].flatten()
        i_nout_ok = main_nout > nout_ini
        main = main[i_nout_ok]
        #x_nout = x_nout[i_nout_ok]
        pos = np.zeros((3,len(main)))
        pos[0,:] = main['x']
        pos[1,:] = main['y']
        pos[2,:] = main['z']


        ## 왜 len(main)?
        mass_ratios_single = np.zeros(len(main))
        nout_inits = np.zeros(len(main))
        #print("log M* ={}".format(np.log10(gal.data["mstar"][0])))

        ## Substitute main["r"] with gal.data["reff"]
        if len(main) < len(gal.smoothed_r):
            main["r"] = gal.smoothed_r[:len(main)]
        elif len(main) > len(gal.smoothed_r):
            main["r"][:len(gal.smoothed_r)] = gal.smoothed_r
        elif len(main) == len(gal.smoothed_r):
            main["r"] = gal.smoothed_r

        for i, nout in enumerate(main['nout']):
            # merger ratio
            # First, check if there are multiple progenitors.
            i_prgs = np.where(atree['desc_id'] == main['id'][i])[0]

            # multiple prgs = merger
            if len(i_prgs) > 1:
                if verbose:
                    print("idx:{}, {} Progenitors at nout = {}".format(main["id"][0], len(i_prgs), nout))

                #print("i, inout_mpgs", i, np.where(gal.nouts == nout)[0])
                #print(gal.data["reff"][i])

                # Mass ratio must be calculated inside get_merger_info. 

                id_prgs = atree['id'][i_prgs]
                mass_prgs = atree['m'][i_prgs]
                #m_r = mass_prgs / max(mass_prgs)

                # Progenitor with maximum mass at the FINAL COALESCENCE is the main progenitor.
                # Others are satellites.
                sats = id_prgs[mass_prgs < max(mass_prgs)]
                
                mass_ratios_now=[]
                nout_inits_now=[]

                # loop over satellites at a given nout.
                for this_sat in sats:
                    n_i_t, mass_this_sat = get_merger_info(main, atree, this_sat,
                                                           dist_gal_scale_in=dist_gal_scale_in,
                                                           dist_gal_scale_out = dist_gal_scale_out,
                                                           do_plot=do_plot,
                                                           ax=ax,
                                                           max_rgal=max_rgal)

                    mass_ratio = mass_this_sat / max(mass_prgs)
                    if do_plot:
                        ax.text(40, 600, "{:.3f}".format(mass_ratio))
                        pdf.savefig()
                        ax.clear()
                    if mass_ratio > min_mass_ratio:
                        nout_inits_now.append(n_i_t)
                        mass_ratios_now.append(mass_ratio)
                if len(nout_inits_now) > 0:
                    i_main_merger = np.argmax(np.array(mass_ratios_now))
                    mass_ratios_single[i] = mass_ratios_now[i_main_merger]
                    nout_inits[i] = nout_inits_now[i_main_merger]
                else:
                    mass_ratios_single[i] = -1
                    nout_inits[i] = -1

                #if verbose:
                #    print(" Mass ratios {} at nout = {}: ".format(m_r, nout_inits[i]))
                #if do_plot:
                #    pdf.savefig()
                    #ax.clear()

            else:
                mass_ratios_single[i] = 0
            ##--------------------------------------------------

        ind_ok = np.where(mass_ratios_single > 0.01)[0]
        if len(ind_ok) > 0:
            # if a satellite oscillates around the host, 
            # it could be identified as multiple mergers with short time interval. 
            # leave only the first passage / merger.
            # No, it doesn't happen in ConsistentTrees.

            #good =[]
            #for i in range(len(ind_ok)-1):
            #    if ind_ok[i+1] > ind_ok[i] + 2:
            #        good.append(ind_ok[i])
            #good.append(ind_ok[-1])
            #ind_ok = good
            mr = 1./mass_ratios_single[ind_ok]

            gal_list.append(idx)
            mr_list.append(mr)
            nout_list.append(main_nout[ind_ok])
            nout_ini_list.append(nout_inits[ind_ok])
            print(idx)
    if do_plot:
        pdf.close()

    inds=[]
    for i, gal in enumerate(mpgs):
        galid = gal.data['idx'][0]
        ind = np.where(galid == gal_list)[0]
        if len(ind) > 0:
            inds.append(i)
            merger = Merger()
            merger.mr = mr_list[ind]
            merger.nout = nout_list[ind]
            merger.nout_ini = nout_ini_list[ind]
            gal.merger = merger
        else:
            gal.merger = None



In [ ]:
def body(clusters,
         dist_gal_scale_in=5,
         dist_gal_scale_out=10,
         dt_before=0.5,
         dt_after=0.5,
         dt_settle=0.5,
         load=False,
         nout_ini = 37,
         filter_small=True,
         min_mass_ratio = 0.05,
         measure_delta_savefig=False,
         find_merger_epoch_plot=False,
         cdir=""):

    suffix = "_{}_{}_{}_{}_{}_{}_{}".format(dist_gal_scale_in,
    dist_gal_scale_out,dt_before,dt_after,dt_settle,nout_ini,
                                           min_mass_ratio)
    if filter_small: 
        suffix = suffix + "_filtered_"

    if load:
        return pickle.load(open("main_prgs_final_augmented" + suffix + ".pickle", 'rb'))
        
    else:
        mpgs = []
        for cluster in clusters:
            print(cluster)
            wdir = base + cluster + '/'

            # Serialize catalogs. -> Only main galaxies
            # main galaxy list
            alltrees = ctu.load_tree(wdir, is_gal=True)
            ad = alltrees.data
            tn = ad[ad['nout'] == nout_fi]

            cat = load_cat(wdir + cdir + 'catalog' + str(nout_fi) + '.pickle')
            #idx_all = [tn['id'][tn['Orig_halo_id'] == id_final][0] for id_final in cat['id']]
            idx_all = cat['idx'][cat["idx"] > 0].astype(int) # why idx are float???

            mpg_tmp = []
            for i, idx in enumerate(idx_all):
                #print(i, idx)

                mpg_tmp.append(MainPrg(ad, idx))
            #    mpg_tmp =[MainPrg(ad, idx) for idx in idx_all]
            for nout in range(nout_ini, nout_fi + 1):
                cat = load_cat(wdir + cdir + 'catalog' + str(nout) + '.pickle')
                for gal in mpg_tmp:
                    gal.set_data(cat, nout)
                    gal.cluster = int(cluster)
        #        print(nout)
            # get rid of galaxies with too short tree.
            mpg_tmp = [gg for gg in mpg_tmp if sum(gg.data["reff"] > 0) > minimum_good_snap]
            for gal in mpg_tmp:
                gal.fill_missing_data()
                gal.clip_non_detection()
                gal.smoothed_lambda_org = mma.smooth(gal.data["lambda_r"], window_len=15)[:-1]
                gal.smoothed_r = mma.smooth(gal.data["reff"], window_len=15)[:-1]
                gal.smoothed_lambda = mma.smooth(l_at_smoothed_r(gal, npix_per_reff=5), window_len=15)[:-1]

            # save for each cluser
            with open(wdir + "main_prgs" + suffix + ".pickle", "wb") as f:
                pickle.dump(mpg_tmp, f)    
                
            # Find_merger_epochs needs smoothed_r
            find_merger_epochs(alltrees,
                               idx_all,
                               mpg_tmp,
                               nout_ini=nout_ini,
                               dist_gal_scale_in=dist_gal_scale_in,
                               dist_gal_scale_out=dist_gal_scale_out,
                               min_mass_ratio = min_mass_ratio,
                               mass_ratio='early',
                               verbose=False,
                               do_plot=find_merger_epoch_plot,
                               max_rgal=40,
                               pdf_fname=str(cluster) + "merger_ratio_epoch" + suffix + ".pdf")
